In [80]:
#基础数据

from tabulate import tabulate
from math import floor

charging_relic = 0.194 #充能绳-充能效率增加19.4%
onwak = 0.05 #翁瓦克-充能效率增加5%
relic_description = ["模式","默认","翁瓦克","充能绳","翁瓦克+充能绳"] #遗器佩戴模式
relic_modes = [0,onwak,charging_relic,onwak+charging_relic] #遗器佩戴模式对应的充能效率


In [81]:
#角色模块
#通用角色模板
class character():

    charging_requirement = 110 #能量上限
    additional_charging = 0 #攻击额外充能
    after_ult_additional_charging = 0 #攻击额外充能：通过伤害型终结技触发的部分
    self_recharge_per_action = 0 #每回合自充能
    charging_efficiency = 0 #充能效率
    A = 20 #释放普攻获得20点能量
    E = 30 #释放战技获得30点能量
    utimate_reset = 5 #释放终结技后回复5点能量
    additional_start_charging = 0 #开局额外获得能量

#佩拉
class Peila(character): 
    
    def __init__(self,skill=0,soul_level=0,trace_level=1):
        self.name = f"佩拉,行迹等级{trace_level}"
        self.charging_requirement = 110 #释放终结技所需能量
        self.soul_level = soul_level
        self.trace_level = trace_level
        self.trace_charging_list = [5,5.5,6,6.5,7,7.5,8.1,8.7,9.3,10,10.5,11] #截取自官方wiki
        self.additional_charging = self.trace_charging_list[self.trace_level-1] #终结技可以造成伤害，补充计算行迹带来的额外充能
        self.after_ult_additional_charging = self.additional_charging    

#银狼
class Yinlang(character):
    
    def __init__(self,skill=0,soul_level=0,trace_level=1):
        self.name = f"{soul_level}命银狼"
        self.charging_requirement = 110 #释放终结技所需能量
        self.soul_level = soul_level
        if self.soul_level >=1: #一魂银狼释放终结技之后，根据目标已有debuff回复能量，最多5*7=35点，此处按35点计算
            self.after_ult_additional_charging = 35
#停云
class Tingyun(character):
    
    def __init__(self,skill=0,soul_level=0,trace_level=1):
        self.name = f"停云,开局释放{skill}次秘技"
        self.skill = skill #开局释放秘技次数
        self.charging_requirement = 130 #释放终结技所需能量
        self.self_recharge_per_action = 5 #额外能力“亨通”，停云的回合开始时，停云回复5点能量，受充能效率影响
        self.additional_start_charging = 50*self.skill #每放一次秘技，开局额外拥有50点能量
    
#艾丝妲
class Aisida(character):
    
    def __init__(self,skill=0,soul_level=0,trace_level=1):
        self.name = f"{soul_level}命艾丝妲"
        self.charging_requirement = 120 #释放终结技所需能量
        self.soul_level = soul_level
        if self.soul_level >=1:  #一魂艾丝妲，战技多一段伤害，充能从30提升到35
            self.E += 5
        if self.soul_level >=4:
            self.charging_efficiency = 0.15  #四魂艾丝妲，被动大于2层时，充能效率提高15%，默认生效
            
#布洛妮娅
class Buluoniya(character):
    
    def __init__(self,skill=0,soul_level=0,trace_level=1):
        self.name = "布洛妮娅"
        self.charging_requirement = 120 #释放终结技所需能量

In [123]:
#光锥模块
#通用光锥模板
class weapon:
        name = "未佩戴充能光锥"
        additional_charging = 0 #攻击额外充能
        after_ult_additional_charging = 0 #攻击额外充能：通过伤害型终结技触发的部分
        charging_efficiency = 0 #充能效率
        additional_start_charging = 0 #开局额外获得能量
        
#不佩戴充能光锥
class Normal(weapon):
    
    def __init__(self,shadow=1):
        self.name = "未佩戴充能光锥"
        self.shadow = shadow

#新手任务开始前：攻击减防敌人获得8点能量，受充能效率影响。
class New_player_mission(weapon): 
    
    def __init__(self,shadow=1):
        self.name = "新手任务开始前"
        self.additional_charging = 8
        self.after_ult_additional_charging = 8
        self.shadow = shadow

#但战斗还未结束：充能效率提升10%
class Battle_not_end(weapon):  
    
    def __init__(self,shadow=1):
        self.name = "但战斗还未结束"
        self.charging_efficiency = 0.1
        self.shadow = shadow
    
#轮契&记忆中的模样：攻击减防敌人获得4-8点能量，吃回能效果
class Memory(weapon):
    
    def __init__(self,shadow=1):
        self.name = f"叠影等级{shadow}轮契/记忆中的模样"
        self.shadow = shadow
        self.additional_charging = 3+self.shadow*1 #叠影1：4点；叠影5：8点
        self.additional_start_charging = self.additional_charging #开局额外获得能量
    

In [124]:
#计算充能函数
def calculate_charging_level(action_charge, action_point, total_additional_charging, character, weapon, relic_mode):

    relic_mode += (weapon.charging_efficiency + character.charging_efficiency) #计算不同仪器佩戴模式的总体充能效率

    base_charging_level = (action_charge +

                           total_additional_charging * action_point +

                           character.self_recharge_per_action * action_point +

                           character.after_ult_additional_charging +

                           weapon.after_ult_additional_charging) #计算基础充能的量，不计充能效率

    charging_level = floor(base_charging_level * (1 + relic_mode) + character.utimate_reset) #计算总体充能的量，计入充能效率和5点终结技充能

    return charging_level

In [125]:
#主模块
def cycle(character=character,weapon=weapon, relic_modes=relic_modes):

    table = []

    total_additional_charging = character.additional_charging + weapon.additional_charging #计算由角色和光锥所带来的总攻击额外充能

 

    for action, action_charge, action_point in [("AA", 2*character.A, 2), ("AE", character.A+character.E, 2),
                                                ("EE", 2*character.E, 2), ("AAA", 3*character.A, 3),
                                                ("AAE", 2*character.A+character.E, 3),("AEE", 2*character.E+character.A, 3),
                                                ("EEE", 3*character.E, 3)]:                                 #7种攻击模式，分别计算

        action_list = [action] 

        for relic_mode in relic_modes: #几种仪器佩戴方式分别计算

            charging_level = calculate_charging_level(action_charge, action_point, total_additional_charging, character, weapon, relic_mode)

            if charging_level >= character.charging_requirement:  #判断可否达成循环

                action_list.append(f'\033[0;32m{charging_level}\033[0m') #达成循环，标绿

            else:

                action_list.append(f'\033[0;31m{charging_level}\033[0m') #未达成循环，标红

        table.append(action_list)

    #开局打战技后是否足以释放终结技
    
    start_list = ["开局打战技后能量"]
    able_list = ["是否支持开局放大"]
    
    for relic_mode in relic_modes:  #几种仪器佩戴方式分别计算
        
        relic_mode += (weapon.charging_efficiency + character.charging_efficiency) #计算不同仪器佩戴模式的总体充能效率
        
        start_level = character.charging_requirement / 2 #开局自带一半充能
        
        start_charging = floor(start_level + 
                               
                         (character.E + total_additional_charging + weapon.additional_start_charging) * (1 + relic_mode) + 
                               
                         character.additional_start_charging) #计算开局释放战技后的能量水平
        
        if start_charging >= character.charging_requirement: #判断可否满足释放终结技的能量需求
            able = "\033[0;32mYes\033[0m"
            start_charging = f'\033[0;32m{start_charging}\033[0m' #满足要求，标绿
        
        else:
            able = "\033[0;31mNo\033[0m"
            start_charging = f'\033[0;31m{start_charging}\033[0m' #未满足要求，标红
        
        start_list.append(start_charging)
        able_list.append(able)  
    
    table.append(start_list)
    table.append(able_list)
        
    #制表
    if weapon.name == "未佩戴充能光锥":
        print(f"以下是<{character.name}><不佩戴充能光锥>的充能明细：")
    
    else:
        print(f"以下是<{character.name}>佩戴光锥<{weapon.name}>的充能明细：")
    
    print(tabulate(table, headers=relic_description,tablefmt='grid'))

In [145]:
#用户输入模块

#声明角色与光锥数组并初始化
character_list = ['1.佩拉','2.银狼','3.停云', '4.艾丝妲','5.布洛妮娅']
weapon_list = ['1.不佩戴充能光锥','2.新手任务开始前','3.但战斗还未结束','4.轮契/记忆中的模样']

#关联函数数组
character_list_index = [Peila,Yinlang,Tingyun,Aisida,Buluoniya]
weapon_list_index = [Normal,New_player_mission,Battle_not_end,Memory]

#需要额外输入参数的角色列表
character_input_skill = [Tingyun]
character_input_trace = [Peila]
character_input_soul = [Yinlang,Aisida]

#需要额外输入参数的光锥列表
weapon_input_shadow = [Memory]

#角色数组的输出
for i in range(len(character_list)):
    print('%s  ' %character_list[i], end='')

#角色输入模块
try :
    c_choice=int(input('\n请选择计算角色:'))
    #输入的判断
    if c_choice>=0 and c_choice<=len(character_list):
        print('%s' %character_list[c_choice-1])
    else:
        print('数字选项输入错误')

except ValueError:
    print('所输入的不是数字，请重新执行！')
    
#加入分割线
print("-----------------------------------------------------------------------------------")
    
#光锥数组的输出
for i in range(len(weapon_list)):
    print('%s  ' %weapon_list[i], end='')
    
#光锥输入模块
try :
    w_choice=int(input('\n请选择角色所佩戴的光锥:'))
    #输入的判断
    if w_choice>=0 and w_choice<=len(weapon_list):
        print('%s' %weapon_list[w_choice-1])
    else:
        print('数字选项输入错误')

except ValueError:
    print('所输入的不是数字，请重新执行！')
    
#加入分割线
print("-----------------------------------------------------------------------------------")
    
#传参模块    
input_character = character_list_index[c_choice-1]
input_weapon = weapon_list_index[w_choice-1]

#初始化参数
input_skill = 0
input_trace_level = 1
input_soul_level = 0
input_shadow = 1

#用户输入
try :
    if input_character in character_input_skill:
        input_skill = int(input('请输入释放秘技的次数：'))

    if input_character in character_input_trace:
        input_trace_level = int(input('请输入行迹等级：'))

    if input_character in character_input_soul:
        input_soul_level = int(input('请输入星魂等级：'))
        
    if input_weapon in weapon_input_shadow:
        input_shadow = int(input('请输入光锥叠影等级：'))

except ValueError:
    print('所输入的不是数字，请重新执行！')

#加入分割线
print("-----------------------------------------------------------------------------------")
#主模块函数
cycle(input_character(skill=input_skill,soul_level=input_soul_level,trace_level=input_trace_level),input_weapon(shadow=input_shadow))

1.佩拉  2.银狼  3.停云  4.艾丝妲  5.布洛妮娅  
请选择计算角色:2
2.银狼
-----------------------------------------------------------------------------------
1.不佩戴充能光锥  2.新手任务开始前  3.但战斗还未结束  4.轮契/记忆中的模样  
请选择角色所佩戴的光锥:2
2.新手任务开始前
-----------------------------------------------------------------------------------
请输入星魂等级：0
-----------------------------------------------------------------------------------
以下是<0命银狼>佩戴光锥<新手任务开始前>的充能明细：
+------------------+--------+----------+----------+-----------------+
| 模式             | 默认   | 翁瓦克   | 充能绳   | 翁瓦克+充能绳   |
+==================+========+==========+==========+=================+
| AA               | 69     | 72       | 81       | 84              |
+------------------+--------+----------+----------+-----------------+
| AE               | 79     | 82       | 93       | 97              |
+------------------+--------+----------+----------+-----------------+
| EE               | 89     | 93       | 105      | 109             |
+------------------+--------+----------+-----

In [121]:
#以下是一些使用实例 
cycle(Peila(trace_level=7),New_player_mission())

以下是<佩拉,行迹等级7>佩戴光锥<新手任务开始前>的充能明细：
+------------------+--------+----------+----------+-----------------+
| 模式             | 默认   | 翁瓦克   | 充能绳   | 翁瓦克+充能绳   |
+==================+========+==========+==========+=================+
| AA               | 93     | 97       | 110      | 114             |
+------------------+--------+----------+----------+-----------------+
| AE               | 103    | 108      | 122      | 127             |
+------------------+--------+----------+----------+-----------------+
| EE               | 113    | 118      | 134      | 139             |
+------------------+--------+----------+----------+-----------------+
| AAA              | 129    | 135      | 153      | 159             |
+------------------+--------+----------+----------+-----------------+
| AAE              | 139    | 146      | 165      | 172             |
+------------------+--------+----------+----------+-----------------+
| AEE              | 149    | 156      | 177      | 184             |
+--

In [132]:
cycle(Aisida(soul_level=4))
cycle(Aisida(soul_level=1),Memory(shadow=5))
cycle(Aisida(soul_level=0),Memory(shadow=5))

以下是<4命艾丝妲><不佩戴充能光锥>的充能明细：
+------------------+--------+----------+----------+-----------------+
| 模式             | 默认   | 翁瓦克   | 充能绳   | 翁瓦克+充能绳   |
+==================+========+==========+==========+=================+
| AA               | 51     | 53       | 58       | 60              |
+------------------+--------+----------+----------+-----------------+
| AE               | 68     | 71       | 78       | 81              |
+------------------+--------+----------+----------+-----------------+
| EE               | 85     | 89       | 99       | 102             |
+------------------+--------+----------+----------+-----------------+
| AAA              | 74     | 77       | 85       | 88              |
+------------------+--------+----------+----------+-----------------+
| AAE              | 91     | 95       | 105      | 109             |
+------------------+--------+----------+----------+-----------------+
| AEE              | 108    | 113      | 125      | 130             |
+---------

In [131]:
cycle(Yinlang(),New_player_mission()) #银狼携带光锥：新手任务开始前
cycle(Yinlang(soul_level=1))

以下是<0命银狼>佩戴光锥<新手任务开始前>的充能明细：
+------------------+--------+----------+----------+-----------------+
| 模式             | 默认   | 翁瓦克   | 充能绳   | 翁瓦克+充能绳   |
+==================+========+==========+==========+=================+
| AA               | 69     | 72       | 81       | 84              |
+------------------+--------+----------+----------+-----------------+
| AE               | 79     | 82       | 93       | 97              |
+------------------+--------+----------+----------+-----------------+
| EE               | 89     | 93       | 105      | 109             |
+------------------+--------+----------+----------+-----------------+
| AAA              | 97     | 101      | 114      | 119             |
+------------------+--------+----------+----------+-----------------+
| AAE              | 107    | 112      | 126      | 131             |
+------------------+--------+----------+----------+-----------------+
| AEE              | 117    | 122      | 138      | 144             |
+------

In [129]:
cycle(Peila(trace_level=5)) #佩拉
cycle(Peila(trace_level=7),New_player_mission()) #佩拉，携带光锥：新手任务开始前

以下是<佩拉,行迹等级5><不佩戴充能光锥>的充能明细：
+------------------+--------+----------+----------+-----------------+
| 模式             | 默认   | 翁瓦克   | 充能绳   | 翁瓦克+充能绳   |
+==================+========+==========+==========+=================+
| AA               | 66     | 69       | 77       | 80              |
+------------------+--------+----------+----------+-----------------+
| AE               | 76     | 79       | 89       | 93              |
+------------------+--------+----------+----------+-----------------+
| EE               | 86     | 90       | 101      | 105             |
+------------------+--------+----------+----------+-----------------+
| AAA              | 93     | 97       | 110      | 114             |
+------------------+--------+----------+----------+-----------------+
| AAE              | 103    | 107      | 122      | 126             |
+------------------+--------+----------+----------+-----------------+
| AEE              | 113    | 118      | 133      | 139             |
+------

In [130]:
cycle(Tingyun(skill=1),Battle_not_end()) #停云
cycle(Tingyun(skill=1))

以下是<停云,开局释放1次秘技>佩戴光锥<但战斗还未结束>的充能明细：
+------------------+--------+----------+----------+-----------------+
| 模式             | 默认   | 翁瓦克   | 充能绳   | 翁瓦克+充能绳   |
+==================+========+==========+==========+=================+
| AA               | 60     | 62       | 69       | 72              |
+------------------+--------+----------+----------+-----------------+
| AE               | 71     | 74       | 82       | 85              |
+------------------+--------+----------+----------+-----------------+
| EE               | 82     | 85       | 95       | 99              |
+------------------+--------+----------+----------+-----------------+
| AAA              | 87     | 91       | 102      | 105             |
+------------------+--------+----------+----------+-----------------+
| AAE              | 98     | 102      | 114      | 119             |
+------------------+--------+----------+----------+-----------------+
| AEE              | 109    | 114      | 127      | 132             |
